In [22]:
import pandas as pd
topic_df=pd.read_csv("/Users/blabbyduck/Desktop/Y3S2/DSA3101/DSA3101-Project/csv_output/App_Responsiveness.csv")

In [23]:
print(topic_df)

    Unnamed: 0                                                key  \
0            0               Difficulties in account registration   
1            1               Difficulties in account registration   
2            2               Difficulties in account registration   
3            3               Difficulties in account registration   
4            4               Difficulties in account registration   
5            5               Difficulties in account registration   
6            6               Difficulties in account registration   
7            7  Issues with loan application and approval process   
8            8  Issues with loan application and approval process   
9            9  Issues with loan application and approval process   
10          10  Issues with loan application and approval process   
11          11      Slow transfer speed and connectivity problems   
12          12      Slow transfer speed and connectivity problems   
13          13     Lack of certain

In [21]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

def analyze_sentiment(topic_df):
    # Initialize the SentimentIntensityAnalyzer
    analyzer = SentimentIntensityAnalyzer()

    # Initialize lists to store data
    review_texts = []
    positive_scores = []
    negative_scores = []
    neutral_scores = []
    compound_scores = []
    nps_indiv = []
    nps_category = []  # New column for NPS categories

    # Perform sentiment analysis and store scores in lists
    for review in topic_df['review']:
        vs = analyzer.polarity_scores(review)

        review_texts.append(review)
        positive_scores.append(vs['pos'])
        negative_scores.append(vs['neg'])
        neutral_scores.append(vs['neu'])
        compound_scores.append(vs['compound'])

        # Map compound scores to nps_indiv based on specified intervals
        if -1 <= vs['compound'] <= -9/11:
            nps_indiv.append(0)
        elif -9/11 < vs['compound'] <= -7/11:
            nps_indiv.append(1)
        elif -7/11 < vs['compound'] <= -5/11:
            nps_indiv.append(2)
        elif -5/11 < vs['compound'] <= -3/11:
            nps_indiv.append(3)
        elif -3/11 < vs['compound'] <= -1/11:
            nps_indiv.append(4)
        elif -1/11 < vs['compound'] <= 1/11:
            nps_indiv.append(5)
        elif 1/11 < vs['compound'] <= 3/11:
            nps_indiv.append(6)
        elif 3/11 < vs['compound'] <= 5/11:
            nps_indiv.append(7)
        elif 5/11 < vs['compound'] <= 7/11:
            nps_indiv.append(8)
        elif 7/11 < vs['compound'] <= 9/11:
            nps_indiv.append(9)
        else:
            nps_indiv.append(10)

        # Map nps_indiv scores to NPS categories
        if nps_indiv[-1] >= 9:  # Promoters
            nps_category.append('Promoter')
        elif nps_indiv[-1] >= 7:  # Passives
            nps_category.append('Passive')
        else:  # Detractors
            nps_category.append('Detractor')

    # Add sentiment scores and NPS categories to the DataFrame
    topic_df['positive_scores'] = positive_scores
    topic_df['negative_scores'] = negative_scores
    topic_df['neutral_scores'] = neutral_scores
    topic_df['compound_scores'] = compound_scores
    topic_df['nps_indiv'] = nps_indiv
    topic_df['nps_category'] = nps_category

    # Calculate Net Promoter Score (NPS) for each issue
    unique_keys = topic_df['key'].unique()
    issues_nps_scores = {}

    for key in unique_keys:
        key_df = topic_df[topic_df['key'] == key]
        label_counts = key_df['nps_category'].value_counts()

        promoter_count = label_counts.get('Promoter', 0)
        detractor_count = label_counts.get('Detractor', 0)
        passive_count = label_counts.get('Passive', 0)
        total_count = promoter_count + detractor_count + passive_count

        if total_count == 0:
            issues_nps_scores[key] = None
        else:
            nps = ((promoter_count - detractor_count) / total_count) * 100
            issues_nps_scores[key] = round(nps, 2)

    issuesNPS = pd.DataFrame(list(issues_nps_scores.items()), columns=['Issue', 'NPS'])
    
    # Calculate topic NPS
    label_counts = topic_df['nps_category'].value_counts()
    promoter_count = label_counts.get('Promoter', 0)
    detractor_count = label_counts.get('Detractor', 0)
    passive_count = label_counts.get('Passive', 0)
    total_count = promoter_count + detractor_count + passive_count

    # Calculate NPS
    if total_count == 0:
        topic_nps = None
    else:
        topic_nps = ((promoter_count - detractor_count) / total_count) * 100
    
    return topic_nps, issuesNPS

# Usage example
topic_nps, issues_nps_df = analyze_sentiment(topic_df)
print("Topic NPS:", topic_nps)
print("Issues NPS:")
print(issues_nps_df)


KeyError: 'key'

      NPS
0  -85.71
1  -25.00
2    0.00
3    0.00
4 -100.00
5 -100.00
6  100.00
7 -100.00
8    0.00
9   35.71
